<a href="https://colab.research.google.com/github/mariarosariolacuna/DownTheRabbitHole/blob/main/Poems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hello!

Day 2 - Document Q&A with RAG using Chroma
Kaggle 5-day Generative AI course!

Two big limitations of LLMs

  - they only "know" the information that they were trained on
  - they have limited input context windows.

RAG- Retrieval Augmented Generation

3 Stages of RAG
  
  - Indexing- happens ahead of time, and allows you to quickly look up relevant information at query-time
  
  - Retrieval- when a query comes in, you retrieve relevant documents, combine them with your instructions and the user's query
  
  - Generation- the LLM generate a tailored answer in natural language using the supplied information.






For this project, I utilized the following public domain poems and biographical backgrounds of each poet via the poets.org website:


A Jelly-Fish by Marianne Moore

Theme in Yellow by Carl Sandburg

My Shadow by Robert Louis Stevenson

The Tyger by William Blake

Wynken, Blynken, and Nod by Eugene Field

##  install ChromaDB and the Gemini API Python SDK.



In [ ]:
%pip install -U -q "google-generativeai>=0.8.3" chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 22.4 MB/s eta 0:00:

In [ ]:
import google.generativeai as genai
from IPython.display import Markdown

## Set up your API key

In [ ]:
from google.colab import userdata


GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Explore available models

In [ ]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)

models/embedding-001
models/text-embedding-004


## Data

In [ ]:
DOCUMENT1 = "A Jelly-Fish. By Marianne Moore. 1887-1972. Visible, invisible, A fluctuating charm, An amber-colored amethyst Inhabits it; your arm Approaches, and It opens and It closes; You have meant To catch it, And it shrivels; You abandon Your intent— It opens, and it Closes and you Reach for it— The blue Surrounding it Grows cloudy, and It floats away From you."

DOCUMENT2 = "Theme in Yellow. By Carl Sandburg. 1878-1967. I spot the hills With yellow balls in autumn. I light the prairie cornfields Orange and tawny gold clusters And I am called pumpkins. On the last of October When dusk is fallen Children join hands And circle round me Singing ghost songs And love to the harvest moon; I am a jack-o'-lantern With terrible teeth And the children know I am fooling."

DOCUMENT3 = "My Shadow. By Robert Louis Stevenson. 1850-1894. I have a little shadow that goes in and out with me, And what can be the use of him is more than I can see. He is very, very like me from the heels up to the head; And I see him jump before me, when I jump into my bed. The funniest thing about him is the way he likes to grow— Not at all like proper children, which is always very slow; For he sometimes shoots up taller like an India-rubber ball, And he sometimes gets so little that there’s none of him at all. He hasn’t got a notion of how children ought to play, And can only make a fool of me in every sort of way. He stays so close beside me, he’s a coward you can see; I’d think shame to stick to nursie as that shadow sticks to me. One morning, very early, before the sun was up, I rose and found the shining dew on every buttercup; But my lazy little shadow, like an arrant sleepy-head, Had stayed at home behind me and was fast asleep in bed."

DOCUMENT4 = "The Tyger. By William Blake. 1757-1827. Tyger! Tyger! burning bright In the forests of the night, What immortal hand or eye Could frame thy fearful symmetry? In what distant deeps or skies Burnt the fire of thine eyes? On what wings dare he aspire? What the hand, dare sieze the fire? And what shoulder, & what art, Could twist the sinews of thy heart? And when thy heart began to beat, What dread hand? & what dread feet? What the hammer? what the chain? In what furnace was thy brain? What the anvil? what dread grasp Dare its deadly terrors clasp? When the stars threw down their spears, And water'd heaven with their tears, Did he smile his work to see? Did he who made the Lamb make thee? Tyger! Tyger! burning bright In the forests of the night, What immortal hand or eye Dare frame thy fearful symmetry?"

DOCUMENT5 = "Wynken, Blynken, and Nod. By Eugene Field. 1850-1895. Wynken, Blynken, and Nod one night Sailed off in a wooden shoe,— Sailed on a river of crystal light Into a sea of dew. 'Where are you going, and what do you wish?' The old moon asked the three. 'We have come to fish for the herring-fish That live in this beautiful sea. Nets of silver and gold have we,' Said Wynken, Blynken, And Nod. The old moon laughed and sang a song, As they rocked in the wooden shoe; And the wind that sped them all night long Ruffled the waves of dew; The little stars were the herring-fish That lived in the beautiful sea. 'Now cast your nets wherever you wish,— Never afraid are we!' So cried the stars to the fishermen three, Wynken, Blynken, And Nod. All night long their nets they threw To the stars in the twinkling foam,— Then down from the skies came the wooden shoe, Bringing the fishermen home: 'Twas all so pretty a sail, it seemed As if it could not be; And some folk thought 'twas a dream they'd dreamed Of sailing that beautiful sea; But I shall name you the fishermen three: Wynken, Blynken, And Nod. Wynken and Blynken are two little eyes, And Nod is a little head, And the wooden shoe that sailed the skies Is a wee one's trundle-bed; So shut your eyes while Mother sings Of wonderful sights that be, And you shall see the beautiful things As you rock in the misty sea Where the old shoe rocked the fishermen three:— Wynken, Blynken, And Nod."

DOCUMENT6 = "Eugene Field. 1850-1895. Eugene Field was born in St. Louis, Missouri, on September 2, 1850. His father was Roswell Martin Field, an attorney who once represented Dred Scott, an African American man known for the 1857 United States Supreme Court case in which he sued for his freedom. After Field’s mother, Frances, died in 1856, he and his brother, Roswell, were sent to Amherst, Massachusetts, to live with Mary Field, their aunt. Field attended Williams College in Williamstown, Massachusetts; Knox College in Galesburg, Illinois; and the University of Missouri in Columbia, but left without graduating. In 1873, he began working at the St. Louis Journal. His humorous column 'Funny Fancies' gained popularity among readers and, in 1880, he moved to Denver, where he worked as managing editor of the Denver Tribune and continued to pen a column. According to the Denver Public Library, 'Eugene was known throughout Denver for his practical jokes. His office at the Denver Tribune included a chair with a false bottom. An unsuspecting person would attempt to sit in the chair and fall to the floor instead.' In 1883, Field moved to Chicago to write a column for the Chicago Daily News. Throughout his career, his columns would occasionally feature his light verse for children, and he became known as the 'Poet of Childhood.' His poems were published in A Little Book of Western Verse (Charles Scribner’s Sons, 1903); The Tribune Primer (Henry A. Dickerman & Son, 1900); and Love-Songs of Childhood (Charles Scribner’s Sons, 1894). Field died on November 4, 1895, in Chicago."

DOCUMENT7 = "William Blake. 1757-1827. William Blake was born in London on November 28, 1757, to James, a hosier, and Catherine Blake. Two of his six siblings died in infancy. From early childhood, Blake spoke of having visions—at four he saw God 'put his head to the window'; around age nine, while walking through the countryside, he saw a tree filled with angels. Although his parents tried to discourage him from 'lying,' they did observe that he was different from his peers and did not force him to attend a conventional school. Instead, he learned to read and write at home. At age ten, Blake expressed a wish to become a painter; so, his parents sent him to drawing school. Two years later, Blake began writing poetry. When he turned fourteen, he apprenticed with an engraver because art school proved too costly. One of Blake’s assignments as apprentice was to sketch the tombs at Westminster Abbey, exposing him to a variety of Gothic styles from which he would draw inspiration throughout his career. After his seven-year term ended, he studied briefly at the Royal Academy. In 1782, Blake married an illiterate woman named Catherine Boucher. Blake taught her to read and write, and also instructed her in draftsmanship. Later, she helped him print the illuminated poetry for which he is remembered today; the couple had no children. In 1784, Blake set up a print shop with friend and former fellow apprentice, James Parker; but this venture failed after several years. For the remainder of his life, Blake made a meager living as an engraver and illustrator for books and magazines. In addition to his wife, Blake also began training his younger brother, Robert, in drawing, painting, and engraving. Robert fell ill during the winter of 1787, having probably succumbed to consumption. As Robert died, Blake saw his brother’s spirit rise up through the ceiling, 'clapping its hands for joy.' He believed that Robert’s spirit continued to visit him and later claimed that in a dream Robert taught him the printing method that he used in Songs of Innocence and other 'illuminated' works. Blake’s first printed work, Poetical Sketches (1783), is a collection of apprentice verse, mostly imitating classical models. The poems protest against war, tyranny, and King George III’s treatment of the American colonies. He published his most popular collection, Songs of Innocence, in 1789 and followed it, in 1794, with Songs of Experience. Some readers interpret Songs of Innocence in a straightforward fashion, considering it primarily a children’s book, but others have found hints at parody or critique in its seemingly naive and simple lyrics. Both books of Songs were printed in an illustrated format reminiscent of illuminated manuscripts. The text and illustrations were printed from copper plates, and each picture was finished by hand in watercolors. Blake was a nonconformist who associated with some of the leading radical thinkers of his day, including Thomas Paine and Mary Wollstonecraft. In defiance of eighteenth-century Neoclassical conventions, he privileged imagination over reason in the creation of both his poetry and images, asserting that ideal forms should be constructed not from observations of nature but from inner visions. He declared in one poem, 'I must create a system or be enslaved by another man’s.' Works such as 'The French Revolution' (1791), 'America, a Prophecy' (1793), 'Visions of the Daughters of Albion' (1793), and 'Europe, a Prophecy' (1794) express his opposition to the English monarchy, and to eighteenth-century political and social tyranny in general. Theological tyranny is the subject of The Book of Urizen (1794). In the prose work The Marriage of Heaven and Hell (1790–93), he satirized the oppressive authority of both church and state, as well as the works of Emanuel Swedenborg, a Swedish philosopher whose ideas once attracted his interest. In 1800, Blake moved to the seacoast town of Felpham, where he lived and worked until 1803 under the patronage of William Hayley. He taught himself Greek, Latin, Hebrew, and Italian, so that he could read classical works in their original language. In Felpham, Blake experienced profound spiritual insights that prepared him for his mature work, the great visionary epics written and etched between about 1804 and 1820. Milton (1804–08); Vala, or The Four Zoas (1797; rewritten after 1800); and Jerusalem (1804–20) have neither traditional plot, characters, rhyme, nor meter. They envision a new and higher kind of innocence—the human spirit triumphing over reason. Blake believed that his poetry could be read and understood by common people, but he was determined not to sacrifice his vision in order to become popular. In 1808, he exhibited some of his watercolors at the Royal Academy and, in May 1809, he exhibited his works at his brother James’s house. Some of those who saw the exhibit praised Blake’s artistry, but others thought the paintings 'hideous' and more than a few called him insane. Blake’s poetry was not well known by the general public, but he was mentioned in A Biographical Dictionary of the Living Authors of Great Britain and Ireland, published in 1816. Samuel Taylor Coleridge, who had been lent a copy of Songs of Innocence and of Experience, considered Blake a 'man of Genius,' and William Wordsworth made his own copies of several songs. Charles Lamb sent a copy of 'The Chimney Sweeper' from Songs of Innocence to James Montgomery for his Chimney-Sweeper’s Friend, and Climbing Boys’ Album (1824), and Robert Southey (who, like Wordsworth, considered Blake insane) attended Blake’s exhibition and included the 'Mad Song' from Poetical Sketches in his miscellany, The Doctor (1834–37). Blake’s final years, spent in great poverty, were cheered by the admiring friendship of a group of younger artists who called themselves 'the Ancients.' In 1818, he met John Linnell, a young artist who helped him financially and also helped to create new interest in his work. It was Linnell who, in 1825, commissioned him to design illustrations for Dante’s Divine Comedy, the cycle of drawings that Blake worked on until his death in 1827."

DOCUMENT8 = "Robert Louis Stevenson. 1850-1894. Born on November 13, 1850, in Edinburgh, Scotland, Robert Louis Balfour Stevenson came from a long line of prominent lighthouse engineers. During his boyhood, he spent holidays with his maternal grandfather, a minister and professor of moral philosophy who shared his love of sermons and storytelling with him. Prone to illness, Stevenson spent many of his early winters in bed, entertained only by his imagination and a great love of reading, especially William Shakespeare, Sir Walter Scott, John Bunyan and The Arabian Nights. Encouraged to follow the family tradition of lighthouse engineering, Stevenson began studies at the University of Edinburgh in 1867, but quickly discovered he preferred a career in literature. To satisfy his father, he acquired a law degree and was admitted to the bar by the time he was twenty-five. Stevenson spent the next four years traveling through Europe, mostly around Paris, publishing essays and articles about his travels. In 1876, he met Fanny Vandegrift Osbourne, a married woman ten years his senior. When she decided to return to San Francisco soon after they met, Stevenson followed, taking the long voyage across the Atlantic and the United States against the advice of his friends and physician. To add to his adventure and inform his writing, he chose to travel in steerage and was near death when he arrived in Monterey, California, in 1879. After being nursed back to health, he continued to San Francisco that winter, though it cost him his health. Osbourne, who had since been divorced, helped him recover. They married the following May. After several months in the U.S. with his wife and her young son, Stevenson brought his new family back to Britain. Frequently sick, he continued to write seriously, producing the bulk of his best-loved work. His first successful novel, Treasure Island was published in 1884, followed by A Child’s Garden of Verses in 1885, and The Strange Case of Dr. Jekyll and Mr. Hyde in 1886. A representative of Neo-Romanticism during the Modernist period of English literature, Stevenson was an incredibly popular and successful writer. Though many leading critics dismissed his work entirely, he was admired by many authors, including Jorge Luis Borges, Ernest Hemingway, Rudyard Kipling, Vladimir Nabokov, and J. M. Barrie. He was also friends with Henry James, who was a vocal supporter his work. Following the death of his father in 1887, Stevenson left again for the U.S. with his family, planning a move to Colorado. Upon landing in New York, however, they decided to spend the winter at Saranac Lake, in the Adirondacks. That summer he chartered a yacht and sailed through eastern and central Pacific, stopping for extended stays among the Hawaiian Islands. In 1890, Stevenson purchased a four hundred-acre estate in Upolu, one of the Samoan islands. He adopted the native name Tusitala (Samoan for 'Story Writer') and soon became immersed in local politics. By 1894, Stevenson had become increasingly depressed, convinced the best of his work was behind him. He wrote that he wished his illnesses would kill him. On the evening of December 3, 1894, he collapsed, possibly due to a cerebral hemorrhage, and died. He is entombed at Mt. Vaea, at a spot overlooking the sea, with a tablet on which his poem 'Requiem' is inscribed. Due in part to the rise of the modernist aesthetic, Stevenson’s work fell out of favor. Criticized by figures such as Virginia Woolf, he was soon remembered only as a contributor to children’s literature. For most of the twentieth century, he was excluded from the Oxford and Norton anthologies of literature entirely, though he is now included. According to an index of translated authors kept by UNESCO, Stevenson is ranked the twenty-fifth most translated author in the world, ahead of fellow Victorians Charles Dickens, Oscar Wilde, and Edgar Allan Poe."

DOCUMENT9 = "Carl Sandburg. 1878-1967. Carl Sandburg was born in Galesburg, Illinois, on January 6, 1878. His parents, August and Clara Johnson, had emigrated to America from the north of Sweden. After encountering several August Johnsons in his job for the railroad, the Sandburg’s father renamed the family. The Sandburgs were very poor; Carl left school at the age of thirteen to work odd jobs, from laying bricks to dishwashing, to help support his family. At seventeen, he traveled west to Kansas as a hobo. He then served eight months in Puerto Rico during the Spanish-American war. While serving, Sandburg met a student at Lombard College, the small school located in Sandburg’s hometown. The young man convinced Sandburg to enroll in Lombard after his return from the war. Sandburg worked his way through school, where he attracted the attention of Professor Philip Green Wright, who not only encouraged Sandburg’s writing, but paid for the publication of his first volume of poetry, a pamphlet called In Reckless Ecstasy (Asgard Press, 1904). While Sandburg attended Lombard for four years, he never received a diploma (he would later receive honorary degrees from Lombard, Knox College, and Northwestern University). After college, Sandburg moved to Milwaukee, where he worked as an advertising writer and a newspaper reporter. While there, he met and married Lillian Steichen (whom he called Paula), sister of the photographer Edward Steichen. A Socialist sympathizer at that point in his life, Sandburg then worked for the Social-Democrat Party in Wisconsin and later acted as secretary to the first Socialist mayor of Milwaukee from 1910 to 1912. The Sandburgs soon moved to Chicago, where Carl became an editorial writer for the Chicago Daily News. Harriet Monroe had just started Poetry: A Magazine of Verse, and began publishing Sandburg’s poems, encouraging him to continue writing in the free-verse, Whitman-like style he had cultivated in college. Monroe liked the poems’ homely speech, which distinguished Sandburg from his predecessors. It was during this period that Sandburg was recognized as a member of the Chicago literary renaissance, which included Ben Hecht, Theodore Dreiser, Sherwood Anderson, and Edgar Lee Masters. He established his reputation with Chicago Poems (Henry Holt and Company, 1916), and then Cornhuskers (Henry Holt and Company, 1918), for which he received the Pulitzer Prize in 1919. Soon after the publication of these volumes Sandburg wrote Smoke and Steel (Harcourt, Brace and Company, 1920), his first prolonged attempt to find beauty in modern industrialism. With these three volumes, Sandburg became known for his free verse poems that portrayed industrial America. In the twenties, he started some of his most ambitious projects, including his study of Abraham Lincoln. From childhood, Sandburg loved and admired the legacy of President Lincoln. For thirty years he sought out and collected material, and gradually began the writing of the six-volume definitive biography of the former president. The twenties also saw Sandburg’s collections of American folklore, the ballads in The American Songbag (Harcourt, Brace and Company, 1927) and New American Songbag (Broadcast Music, Inc., 1950), and books for children. These later volumes contained pieces collected from brief tours across America which Sandburg took each year, playing his banjo or guitar, singing folk-songs, and reciting poems. In the 1930s, Sandburg continued his celebration of America with Mary Lincoln, Wife and Widow (Harcourt, Brace and Company, 1932); The People, Yes (Harcourt, Brace and Company, 1936); and the second part of his Lincoln biography, Abraham Lincoln: The War Years (Harcourt, Brace and Company, 1939), for which he was awarded the Pulitzer Prize. He received a second Pulitzer Prize for his Complete Poems (Harcourt, Brace and Company) in 1950. His final volumes of verse were Harvest Poems, 1910–1960 (Harcourt, Brace and Company, 1960) and Honey and Salt (Harcourt, Brace and Company, 1963). Carl Sandburg died on July 22, 1967. Sandburg was inducted to the American Poets’ Corner at the Cathedral of St. John the Divine in New York in 2018."

DOCUMENT10 = "Marianne Moore. 1887-1972. Born near St. Louis, Missouri, on November 15, 1887, Marianne Moore was raised in the home of her grandfather, a Presbyterian pastor. After her grandfather’s death, in 1894, Moore and her family stayed with other relatives, and in 1896 they moved to Carlisle, Pennsylvania. She attended Bryn Mawr College and received her BA in 1909. Following graduation, Moore studied typing at Carlisle Commercial College, and from 1911 to 1915 she was employed as a school teacher at the Carlisle Indian School. In 1918, Moore and her mother moved to New York City, and in 1921, she became an assistant at the New York Public Library. She began to meet other poets, such as William Carlos Williams and Wallace Stevens, and to contribute to the Dial, a prestigious literary magazine. She served as acting editor of the Dial from 1925 to 1929. Along with the work of such other members of the Imagist movement as Ezra Pound, Williams, and H.D., Moore’s poems were published in The Egoist, an English magazine, beginning in 1915. In 1921, H.D. published Moore’s first book, Poems (The Egoist Press, 1921), without her knowledge. Moore was widely recognized for her work; among her many honors were the Bollingen prize, the National Book Award, and the Pulitzer Prize. She wrote with the freedom characteristic of the other Modernist poets, often incorporating quotes from other sources into the text, yet her use of language was always extraordinarily condensed and precise, capable of suggesting a variety of ideas and associations within a single, compact image. In his 1925 essay 'Marianne Moore,' William Carlos Williams wrote about Moore’s signature mode, the vastness of the particular: 'So that in looking at some apparently small object, one feels the swirl of great events.' She was particularly fond of animals, and much of her imagery is drawn from the natural world. She was also a great fan of professional baseball and an admirer of Muhammed Ali, for whom she wrote the liner notes to his record, I Am the Greatest! Deeply attached to her mother, she lived with her until Mrs. Moore’s death in 1947. Marianne Moore died in New York City on February 5, 1972."


documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4, DOCUMENT5, DOCUMENT6, DOCUMENT7, DOCUMENT8, DOCUMENT9, DOCUMENT10]

## Creating the embedding database with ChromaDB

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        response = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        return response["embedding"]

In [ ]:
# Now create a Chroma database client that uses the GeminiEmbeddingFunction and populate the database with the documents you defined above.

import chromadb

DB_NAME = "poemsdb"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

In [ ]:
# Confirm that the data was inserted by looking at the database.

db.count()
# You can peek at the data too.
# db.peek(1)

10

## Retrieval: Find relevant documents

In [ ]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "tell me about the poem The Shadow?"

result = db.query(query_texts=[query], n_results=1)
[[passage]] = result["documents"]

Markdown(passage)

My Shadow. By Robert Louis Stevenson. 1850-1894. I have a little shadow that goes in and out with me, And what can be the use of him is more than I can see. He is very, very like me from the heels up to the head; And I see him jump before me, when I jump into my bed. The funniest thing about him is the way he likes to grow— Not at all like proper children, which is always very slow; For he sometimes shoots up taller like an India-rubber ball, And he sometimes gets so little that there’s none of him at all. He hasn’t got a notion of how children ought to play, And can only make a fool of me in every sort of way. He stays so close beside me, he’s a coward you can see; I’d think shame to stick to nursie as that shadow sticks to me. One morning, very early, before the sun was up, I rose and found the shining dew on every buttercup; But my lazy little shadow, like an arrant sleepy-head, Had stayed at home behind me and was fast asleep in bed. This poem is in the public domain.

## Augmented generation: Answer the question

In [ ]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
However, you are talking to a non-literary audience, so be sure to break down complicated concepts and
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-literary audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: tell me about the poem The Shadow?
PASSAGE: My Shadow. By Robert Louis Stevenson. 1850-1894. I have a little shadow that goes in and out with me, And what can be the use of him is more than I can see. He is very, very like me from the heels up to the head; And I see him jump before me, when I jump into my bed. The funniest thing about him is the way he likes to grow— Not at all like proper children, which is always very slow; For he sometimes shoots up taller like an India-rubber ball, And he sometimes gets so little that there’s none of him at a

In [ ]:
# Now use the generate_content method to to generate an answer to the question.

model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer = model.generate_content(prompt)
Markdown(answer.text)

"The Shadow," written by Robert Louis Stevenson between 1850 and 1894, is a whimsical poem about a child's playful observation of their shadow.  The poem personifies the shadow, describing it as a quirky companion that mimics the child's actions but in a strange and unpredictable way. The shadow's size changes unexpectedly, sometimes growing enormously and other times disappearing completely. It also acts in a cowardly manner, always staying close to the child. The child finds the shadow's behavior amusing and even frustrating at times, highlighting the childlike wonder and imagination at play.  The poem ends with a humorous detail about the shadow sleeping in while the child wakes to see the morning dew.


## Example 2

In [ ]:

# Search the Chroma DB using the specified query.
query = "tell me about the poem Wynken, Blynken, and Nod?"

result = db.query(query_texts=[query], n_results=1)
[[passage]] = result["documents"]

Markdown(passage)

Wynken, Blynken, and Nod. By Eugene Field. 1850-1895. Wynken, Blynken, and Nod one night Sailed off in a wooden shoe,— Sailed on a river of crystal light Into a sea of dew. 'Where are you going, and what do you wish?' The old moon asked the three. 'We have come to fish for the herring-fish That live in this beautiful sea. Nets of silver and gold have we,' Said Wynken, Blynken, And Nod. The old moon laughed and sang a song, As they rocked in the wooden shoe; And the wind that sped them all night long Ruffled the waves of dew; The little stars were the herring-fish That lived in the beautiful sea. 'Now cast your nets wherever you wish,— Never afraid are we!' So cried the stars to the fishermen three, Wynken, Blynken, And Nod. All night long their nets they threw To the stars in the twinkling foam,— Then down from the skies came the wooden shoe, Bringing the fishermen home: 'Twas all so pretty a sail, it seemed As if it could not be; And some folk thought 'twas a dream they'd dreamed Of sailing that beautiful sea; But I shall name you the fishermen three: Wynken, Blynken, And Nod. Wynken and Blynken are two little eyes, And Nod is a little head, And the wooden shoe that sailed the skies Is a wee one's trundle-bed; So shut your eyes while Mother sings Of wonderful sights that be, And you shall see the beautiful things As you rock in the misty sea Where the old shoe rocked the fishermen three:— Wynken, Blynken, And Nod.

In [ ]:

passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
However, you are talking to a non-literary audience, so be sure to break down complicated concepts and
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-literary audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: tell me about the poem Wynken, Blynken, and Nod?
PASSAGE: Wynken, Blynken, and Nod. By Eugene Field. 1850-1895. Wynken, Blynken, and Nod one night Sailed off in a wooden shoe,— Sailed on a river of crystal light Into a sea of dew. 'Where are you going, and what do you wish?' The old moon asked the three. 'We have come to fish for the herring-fish That live in this beautiful sea. Nets of silver and gold have we,' Said Wynken, Blynken, And Nod. The old moon laughed and sang a song, As they rocked in the wooden shoe; And the wind that sped them all 

In [ ]:
# Now use the generate_content method to to generate an answer to the question.

model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer = model.generate_content(prompt)
Markdown(answer.text)

"Wynken, Blynken, and Nod" is a children's poem written by Eugene Field (1850-1895).  The poem tells a whimsical story of three little fishermen – Wynken, Blynken, and Nod – who sail in a wooden shoe across a river of light and a sea of dew to fish for stars.  The old moon guides them, and the wind gently rocks their shoe.  The poem ends with a sweet revelation: Wynken and Blynken are actually the child's eyes, Nod is their head, and the wooden shoe is their bed!  It's a lovely poem about the magical world of dreams and the comforting presence of a mother's song.


## Example 3

In [ ]:
# Search the Chroma DB using the specified query.
query = "who is william blake?"

result = db.query(query_texts=[query], n_results=1)
[[passage]] = result["documents"]

Markdown(passage)

William Blake. 1757-1827. William Blake was born in London on November 28, 1757, to James, a hosier, and Catherine Blake. Two of his six siblings died in infancy. From early childhood, Blake spoke of having visions—at four he saw God 'put his head to the window'; around age nine, while walking through the countryside, he saw a tree filled with angels. Although his parents tried to discourage him from 'lying,' they did observe that he was different from his peers and did not force him to attend a conventional school. Instead, he learned to read and write at home. At age ten, Blake expressed a wish to become a painter; so, his parents sent him to drawing school. Two years later, Blake began writing poetry. When he turned fourteen, he apprenticed with an engraver because art school proved too costly. One of Blake’s assignments as apprentice was to sketch the tombs at Westminster Abbey, exposing him to a variety of Gothic styles from which he would draw inspiration throughout his career. After his seven-year term ended, he studied briefly at the Royal Academy. In 1782, Blake married an illiterate woman named Catherine Boucher. Blake taught her to read and write, and also instructed her in draftsmanship. Later, she helped him print the illuminated poetry for which he is remembered today; the couple had no children. In 1784, Blake set up a print shop with friend and former fellow apprentice, James Parker; but this venture failed after several years. For the remainder of his life, Blake made a meager living as an engraver and illustrator for books and magazines. In addition to his wife, Blake also began training his younger brother, Robert, in drawing, painting, and engraving. Robert fell ill during the winter of 1787, having probably succumbed to consumption. As Robert died, Blake saw his brother’s spirit rise up through the ceiling, 'clapping its hands for joy.' He believed that Robert’s spirit continued to visit him and later claimed that in a dream Robert taught him the printing method that he used in Songs of Innocence and other 'illuminated' works. Blake’s first printed work, Poetical Sketches (1783), is a collection of apprentice verse, mostly imitating classical models. The poems protest against war, tyranny, and King George III’s treatment of the American colonies. He published his most popular collection, Songs of Innocence, in 1789 and followed it, in 1794, with Songs of Experience. Some readers interpret Songs of Innocence in a straightforward fashion, considering it primarily a children’s book, but others have found hints at parody or critique in its seemingly naive and simple lyrics. Both books of Songs were printed in an illustrated format reminiscent of illuminated manuscripts. The text and illustrations were printed from copper plates, and each picture was finished by hand in watercolors. Blake was a nonconformist who associated with some of the leading radical thinkers of his day, including Thomas Paine and Mary Wollstonecraft. In defiance of eighteenth-century Neoclassical conventions, he privileged imagination over reason in the creation of both his poetry and images, asserting that ideal forms should be constructed not from observations of nature but from inner visions. He declared in one poem, 'I must create a system or be enslaved by another man’s.' Works such as 'The French Revolution' (1791), 'America, a Prophecy' (1793), 'Visions of the Daughters of Albion' (1793), and 'Europe, a Prophecy' (1794) express his opposition to the English monarchy, and to eighteenth-century political and social tyranny in general. Theological tyranny is the subject of The Book of Urizen (1794). In the prose work The Marriage of Heaven and Hell (1790–93), he satirized the oppressive authority of both church and state, as well as the works of Emanuel Swedenborg, a Swedish philosopher whose ideas once attracted his interest. In 1800, Blake moved to the seacoast town of Felpham, where he lived and worked until 1803 under the patronage of William Hayley. He taught himself Greek, Latin, Hebrew, and Italian, so that he could read classical works in their original language. In Felpham, Blake experienced profound spiritual insights that prepared him for his mature work, the great visionary epics written and etched between about 1804 and 1820. Milton (1804–08); Vala, or The Four Zoas (1797; rewritten after 1800); and Jerusalem (1804–20) have neither traditional plot, characters, rhyme, nor meter. They envision a new and higher kind of innocence—the human spirit triumphing over reason. Blake believed that his poetry could be read and understood by common people, but he was determined not to sacrifice his vision in order to become popular. In 1808, he exhibited some of his watercolors at the Royal Academy and, in May 1809, he exhibited his works at his brother James’s house. Some of those who saw the exhibit praised Blake’s artistry, but others thought the paintings 'hideous' and more than a few called him insane. Blake’s poetry was not well known by the general public, but he was mentioned in A Biographical Dictionary of the Living Authors of Great Britain and Ireland, published in 1816. Samuel Taylor Coleridge, who had been lent a copy of Songs of Innocence and of Experience, considered Blake a 'man of Genius,' and William Wordsworth made his own copies of several songs. Charles Lamb sent a copy of 'The Chimney Sweeper' from Songs of Innocence to James Montgomery for his Chimney-Sweeper’s Friend, and Climbing Boys’ Album (1824), and Robert Southey (who, like Wordsworth, considered Blake insane) attended Blake’s exhibition and included the 'Mad Song' from Poetical Sketches in his miscellany, The Doctor (1834–37). Blake’s final years, spent in great poverty, were cheered by the admiring friendship of a group of younger artists who called themselves 'the Ancients.' In 1818, he met John Linnell, a young artist who helped him financially and also helped to create new interest in his work. It was Linnell who, in 1825, commissioned him to design illustrations for Dante’s Divine Comedy, the cycle of drawings that Blake worked on until his death in 1827.

In [ ]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
However, you are talking to a non-literary audience, so be sure to break down complicated concepts and
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""
print(prompt)


You are a helpful and informative bot that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
However, you are talking to a non-literary audience, so be sure to break down complicated concepts and
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: who is william blake?
PASSAGE: William Blake. 1757-1827. William Blake was born in London on November 28, 1757, to James, a hosier, and Catherine Blake. Two of his six siblings died in infancy. From early childhood, Blake spoke of having visions—at four he saw God 'put his head to the window'; around age nine, while walking through the countryside, he saw a tree filled with angels. Although his parents tried to discourage him from 'lying,' they did observe that he was different from his peers and did not force him to attend a conventional school. In

In [ ]:
# Now use the generate_content method to to generate an answer to the question.

model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer = model.generate_content(prompt)
Markdown(answer.text)

William Blake (1757-1827) was an English poet, painter, and printmaker.  Born in London, he had unusual experiences from a young age, reporting visions of God and angels.  Instead of traditional schooling, he learned at home and later apprenticed as an engraver, developing skills that would be crucial to his unique artistic style.  He married Catherine Boucher, whom he taught to read and write, and they collaborated on his illuminated poetry.  Blake was a radical thinker who challenged conventional artistic and political norms of his time, creating works that celebrated imagination over reason and expressed his opposition to tyranny.  His famous works include *Songs of Innocence and Experience* and visionary epics like *Jerusalem*. While not widely recognized during his lifetime, Blake's work has since gained significant acclaim, influencing generations of artists and thinkers.


# Example 4

In [ ]:
# Search the Chroma DB using the specified query.
query = "who is robert louis steventson?"

result = db.query(query_texts=[query], n_results=1)
[[passage]] = result["documents"]

Markdown(passage)

Robert Louis Stevenson. 1850-1894. Born on November 13, 1850, in Edinburgh, Scotland, Robert Louis Balfour Stevenson came from a long line of prominent lighthouse engineers. During his boyhood, he spent holidays with his maternal grandfather, a minister and professor of moral philosophy who shared his love of sermons and storytelling with him. Prone to illness, Stevenson spent many of his early winters in bed, entertained only by his imagination and a great love of reading, especially William Shakespeare, Sir Walter Scott, John Bunyan and The Arabian Nights. Encouraged to follow the family tradition of lighthouse engineering, Stevenson began studies at the University of Edinburgh in 1867, but quickly discovered he preferred a career in literature. To satisfy his father, he acquired a law degree and was admitted to the bar by the time he was twenty-five. Stevenson spent the next four years traveling through Europe, mostly around Paris, publishing essays and articles about his travels. In 1876, he met Fanny Vandegrift Osbourne, a married woman ten years his senior. When she decided to return to San Francisco soon after they met, Stevenson followed, taking the long voyage across the Atlantic and the United States against the advice of his friends and physician. To add to his adventure and inform his writing, he chose to travel in steerage and was near death when he arrived in Monterey, California, in 1879. After being nursed back to health, he continued to San Francisco that winter, though it cost him his health. Osbourne, who had since been divorced, helped him recover. They married the following May. After several months in the U.S. with his wife and her young son, Stevenson brought his new family back to Britain. Frequently sick, he continued to write seriously, producing the bulk of his best-loved work. His first successful novel, Treasure Island was published in 1884, followed by A Child’s Garden of Verses in 1885, and The Strange Case of Dr. Jekyll and Mr. Hyde in 1886. A representative of Neo-Romanticism during the Modernist period of English literature, Stevenson was an incredibly popular and successful writer. Though many leading critics dismissed his work entirely, he was admired by many authors, including Jorge Luis Borges, Ernest Hemingway, Rudyard Kipling, Vladimir Nabokov, and J. M. Barrie. He was also friends with Henry James, who was a vocal supporter his work. Following the death of his father in 1887, Stevenson left again for the U.S. with his family, planning a move to Colorado. Upon landing in New York, however, they decided to spend the winter at Saranac Lake, in the Adirondacks. That summer he chartered a yacht and sailed through eastern and central Pacific, stopping for extended stays among the Hawaiian Islands. In 1890, Stevenson purchased a four hundred-acre estate in Upolu, one of the Samoan islands. He adopted the native name Tusitala (Samoan for 'Story Writer') and soon became immersed in local politics. By 1894, Stevenson had become increasingly depressed, convinced the best of his work was behind him. He wrote that he wished his illnesses would kill him. On the evening of December 3, 1894, he collapsed, possibly due to a cerebral hemorrhage, and died. He is entombed at Mt. Vaea, at a spot overlooking the sea, with a tablet on which his poem 'Requiem' is inscribed. Due in part to the rise of the modernist aesthetic, Stevenson’s work fell out of favor. Criticized by figures such as Virginia Woolf, he was soon remembered only as a contributor to children’s literature. For most of the twentieth century, he was excluded from the Oxford and Norton anthologies of literature entirely, though he is now included. According to an index of translated authors kept by UNESCO, Stevenson is ranked the twenty-fifth most translated author in the world, ahead of fellow Victorians Charles Dickens, Oscar Wilde, and Edgar Allan Poe.

In [ ]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
However, you are talking to a gamer teenager audience, so be sure to break down complicated concepts and
strike a friendly and converstional tone to make them interested. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
However, you are talking to a gamer teenager audience, so be sure to break down complicated concepts and
strike a friendly and converstional tone to make them interested. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: who is robert louis steventson?
PASSAGE: Robert Louis Stevenson. 1850-1894. Born on November 13, 1850, in Edinburgh, Scotland, Robert Louis Balfour Stevenson came from a long line of prominent lighthouse engineers. During his boyhood, he spent holidays with his maternal grandfather, a minister and professor of moral philosophy who shared his love of sermons and storytelling with him. Prone to illness, Stevenson spent many of his early winters in bed, entertained only by his imagination and a great love of reading, especiall

In [ ]:
# Now use the generate_content method to to generate an answer to the question.

model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer = model.generate_content(prompt)
Markdown(answer.text)

Hey gamers!  So, Robert Louis Stevenson?  Think of him as the ultimate Victorian-era level-up boss. Born in 1850 in Scotland, this dude was a total literary legend, even if some later critics tried to nerf him.  His family was all about lighthouse engineering, super serious stuff, but Stevenson? He was more into storytelling.  He spent his childhood battling illness and devouring books like Shakespeare, Sir Walter Scott, and *The Arabian Nights* – talk about serious RPG inspiration!

He *tried* law school to please his dad, but secretly, he was busy leveling up his writing skills.  He travelled all over Europe, even braving a super dangerous trip across the Atlantic to meet his wife – a total badass move.  He wrote tons of essays and articles along the way. This dude was all about adventure!

His fame really exploded with *Treasure Island* in 1884 – seriously epic loot! Then came *A Child’s Garden of Verses* and *Dr. Jekyll and Mr. Hyde*, both instant classics. While some critics hated his work, he was hugely popular with other authors like Hemingway and Borges – who are like, the ultimate writer gods!

Later in life, he moved to the Samoan Islands, got a cool nickname ("Storyteller"), and basically lived the tropical island life until he died in 1894.  Even though he wasn't always super high on the literary charts, he's still ranked as one of the most translated authors ever – a true testament to how awesome his stories are. So yeah, Robert Louis Stevenson: a true legend of adventure and literature!
